In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=743c51c9d33d656fdbf8eab3f63eb2d14a0278f0224c34cd053a6da9d7a14558
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd /content/drive/My Drive/Spark/Advanced Spark/Use Broadcast with Movies

/content/drive/My Drive/Spark/Advanced Spark/Use Broadcast with Movies


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructType, StructField, IntegerType, LongType
import codecs

In [8]:
def loadMovieNames():
    movieNames = {}
    with codecs.open("/content/drive/My Drive/Spark/Advanced Spark/Use Broadcast with Movies/ml-100k/u.item", "r", encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

In [9]:
spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

In [10]:
nameDict = spark.sparkContext.broadcast(loadMovieNames())

In [11]:
schema = StructType([StructField("userID", IntegerType(), True),
                     StructField("movieID", IntegerType(), True),
                     StructField("rating", IntegerType(), True),
                     StructField("timestamp", LongType(), True)])

In [12]:
moviesDF = spark.read.option("sep", "\t").schema(schema).csv("/content/drive/My Drive/Spark/Advanced Spark/Use Broadcast with Movies/ml-100k/u.data")

In [13]:
movieCounts = moviesDF.groupBy("movieID").count()

In [14]:
def lookupName(movieID):
    return nameDict.value[movieID]

In [15]:
lookupNameUDF = func.udf(lookupName)

In [17]:
# add a movie title column using our new udf
moviesWithNames = movieCounts.withColumn("movieTitle", lookupNameUDF(func.col("movieID")))

In [18]:
# sort the results
sortedMoviesWithNames = moviesWithNames.orderBy(func.desc("count"))

In [19]:
# grab the top 10
sortedMoviesWithNames.show(10, False)

+-------+-----+-----------------------------+
|movieID|count|movieTitle                   |
+-------+-----+-----------------------------+
|50     |583  |Star Wars (1977)             |
|258    |509  |Contact (1997)               |
|100    |508  |Fargo (1996)                 |
|181    |507  |Return of the Jedi (1983)    |
|294    |485  |Liar Liar (1997)             |
|286    |481  |English Patient, The (1996)  |
|288    |478  |Scream (1996)                |
|1      |452  |Toy Story (1995)             |
|300    |431  |Air Force One (1997)         |
|121    |429  |Independence Day (ID4) (1996)|
+-------+-----+-----------------------------+
only showing top 10 rows

